In [ ]:
# .\.venv\Scripts\Activate.ps1
# pip install -r requirements.txt

In [2]:
import openai
import json
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Get the API key from the environment variables
api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key was found and raise an error if not
if not api_key:
    raise ValueError("OpenAI API key not found. Please make sure OPENAI_API_KEY is set in your .env file.")


# Define the part we want to machine
# This is a natural language description that serves as the input to our process
part_description = """
Part Name: Simple Mounting Bracket
Material: Aluminum 6061-T6
Dimensions: Rectangular block, 100mm long, 50mm wide, 20mm thick.
Features:
1. The top surface (100x50mm) needs to be faced to achieve a clean, flat finish.
2. Four (4) M6 through-holes need to be drilled. The holes are located at each corner, 10mm in from each edge.
The raw stock is slightly larger than the final dimensions.
"""

# Print a confirmation message
print("Part description has been defined.")

Part description has been defined.


In [4]:
def get_process_plan_from_llm(description: str) -> str:
    """
    Calls the LLM API to generate a suggested process plan based on the part description.

    Args:
        description: A natural language description of the part.

    Returns:
        The raw string returned by the LLM, expected to be in JSON format, or None if an error occurs.
    """
    # ------------------ Actual API Call Code (Start) ------------------
    # Initialize the OpenAI client using the previously configured API key
    client = openai.OpenAI() 
    
    # Construct a detailed prompt for the LLM
    prompt = f"""
    You are an expert CNC machinist and process planner. Your task is to generate a step-by-step
    process plan for manufacturing a mechanical part based on the provided description.
    
    Part Description:
    ---
    {description}
    ---
    
    Instructions:
    Generate a detailed process plan. The output MUST be a single, valid JSON object that contains one key: "plan".
    The value of "plan" must be an array of objects.
    Each object in the array represents one machining step and must contain the following keys:
    - "step": (Integer) The sequence number of the operation.
    - "operation": (String) The name of the machining operation (e.g., "Setup", "Facing", "Drilling").
    - "tool_description": (String) A description of the tool to be used (e.g., "10mm 4-Flute End Mill", "M6 Tap").
    - "tool_diameter_mm": (Float or null) The diameter of the tool in millimeters.
    - "spindle_speed_rpm": (Integer or null) The recommended spindle speed in RPM for the specified material.
    - "feed_rate_mm_min": (Integer or null) The recommended feed rate in mm/min for the specified material.
    - "notes": (String) Important details, setup instructions, or comments for the machinist.
    
    Do not add any explanation, markdown, or text outside of the JSON object.
    """
    
    # Use a try-except block to handle potential API errors
    try:
        print("Sending a real API request to OpenAI, please wait...")
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an expert CNC machinist and process planner designed to output JSON."},
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"} # Use JSON mode
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred during the API call: {e}")
        return None
    # ------------------ Actual API Call Code (End) ------------------

# Execute the LLM call
print("\n1. Starting to generate the process plan from the LLM...")
llm_response_str = get_process_plan_from_llm(part_description)

if llm_response_str:
    print("LLM response received successfully.")
    # --- ADD THIS LINE FOR DEBUGGING ---
    print("\n--- RAW LLM RESPONSE ---")
    print(llm_response_str)
    print("------------------------\n")
    # ---------------------------------
else:
    print("Failed to get a response from the LLM. Please check the error message above.")
    
# Subsequent code (parsing, validation, etc.) will process llm_response_str
# If the API call fails, subsequent steps may need to be adjusted to handle None values

# ------------------ Simulated LLM JSON response string (for demonstration) ------------------
    # print("Note: Using simulated LLM response for demonstration.")
    # mock_llm_output = """
    # {
    #   "plan": [
    #     {
    #       "step": 1,
    #       "operation": "Setup",
    #       "tool_description": "N/A",
    #       "tool_diameter_mm": null,
    #       "spindle_speed_rpm": null,
    #       "feed_rate_mm_min": null,
    #       "notes": "Secure the raw stock in a vise. Ensure the top surface is accessible. Use parallels to lift the part for through-drilling clearance."
    #     },
    #     {
    #       "step": 2,
    #       "operation": "Facing",
    #       "tool_description": "50mm Face Mill with carbide inserts",
    #       "tool_diameter_mm": 50.0,
    #       "spindle_speed_rpm": 3000,
    #       "feed_rate_mm_min": 600,
    #       "notes": "Face the top surface (100x50mm) to achieve final thickness of 20mm. Take a light finishing pass for a good surface finish."
    #     },
    #     {
    #       "step": 3,
    #       "operation": "Center Drilling",
    #       "tool_description": "Center Drill #3",
    #       "tool_diameter_mm": 4.0,
    #       "spindle_speed_rpm": 2500,
    #       "feed_rate_mm_min": 150,
    #       "notes": "Center drill the four M6 hole locations to provide a precise starting point for the drill bit."
    #     },
    #     {
    #       "step": 4,
    #       "operation": "Drilling",
    #       "tool_description": "5.0mm HSS Drill Bit",
    #       "tool_diameter_mm": 5.0,
    #       "spindle_speed_rpm": 9000,
    #       "feed_rate_mm_min": 300,
    #       "notes": "Drill the four M6 through-holes. A 5.0mm drill is standard for an M6 tap. High RPM is a typo, should be lower for HSS."
    #     },
    #     {
    #       "step": 5,
    #       "operation": "Deburring",
    #       "tool_description": "Deburring Tool",
    #       "tool_diameter_mm": 6.0,
    #       "spindle_speed_rpm": 800,
    #       "feed_rate_mm_min": null,
    #       "notes": "Lightly deburr the edges of the freshly drilled holes on the top surface. This could also be a manual operation."
    #     },
    #     {
    #       "step": 6,
    #       "operation": "Final Inspection",
    #       "tool_description": "Calipers / Micrometer",
    #       "tool_diameter_mm": null,
    #       "spindle_speed_rpm": null,
    #       "feed_rate_mm_min": null,
    #       "notes": "Verify all dimensions, hole locations, and overall finish meet the requirements."
    #     }
    #   ]
    # }
    # """
    # return mock_llm_output


1. Starting to generate the process plan from the LLM...
Sending a real API request to OpenAI, please wait...
LLM response received successfully.

--- RAW LLM RESPONSE ---
{
  "plan": [
    {
      "step": 1,
      "operation": "Setup",
      "tool_description": "Vise or fixture for clamping",
      "tool_diameter_mm": null,
      "spindle_speed_rpm": null,
      "feed_rate_mm_min": null,
      "notes": "Secure the raw stock in a suitable vise or fixture, ensuring it is stable and properly aligned."
    },
    {
      "step": 2,
      "operation": "Facing",
      "tool_description": "50mm Face Mill",
      "tool_diameter_mm": 50,
      "spindle_speed_rpm": 1200,
      "feed_rate_mm_min": 200,
      "notes": "Face the top surface to achieve a clean, flat finish. Ensure even feed across the surface."
    },
    {
      "step": 3,
      "operation": "Drilling",
      "tool_description": "6mm Twist Drill Bit",
      "tool_diameter_mm": 6,
      "spindle_speed_rpm": 1000,
      "feed_rate_

In [5]:
def post_process_response(response_str: str) -> list:
    """
    Parses the JSON string returned by the LLM and converts it into a Python list.

    Args:
        response_str: The JSON string returned by the LLM.

    Returns:
        A Python list containing the process steps, or an empty list if parsing fails.
    """
    if not response_str:
        print("Error: The received response is empty.")
        return []
    try:
        # The LLM might return a top-level object containing the plan, so we need to extract the list.
        data = json.loads(response_str)
        if "plan" in data and isinstance(data["plan"], list):
            return data["plan"]
        else:
            # This also handles cases where the LLM returns a list directly.
            return data
    except json.JSONDecodeError:
        print("Error: Failed to parse the LLM response as JSON. Please check its format.")
        # More complex logic could be added here, like trying to fix incomplete JSON with regular expressions.
        return []

print("\n2. Post-processing the LLM response...")
parsed_plan = post_process_response(llm_response_str)
print("Post-processing complete. The JSON string has been converted to a Python list.")
# --- ADD THIS LINE FOR DEBUGGING ---
print("\n--- PARSED DATA STRUCTURE ---")
print(f"Type of parsed_plan: {type(parsed_plan)}")
print(parsed_plan)
print("---------------------------\n")
# ---------------------------------
# print("\nParsed Plan:\n", parsed_plan)


2. Post-processing the LLM response...
Post-processing complete. The JSON string has been converted to a Python list.

--- PARSED DATA STRUCTURE ---
Type of parsed_plan: <class 'list'>
[{'step': 1, 'operation': 'Setup', 'tool_description': 'Vise or fixture for clamping', 'tool_diameter_mm': None, 'spindle_speed_rpm': None, 'feed_rate_mm_min': None, 'notes': 'Secure the raw stock in a suitable vise or fixture, ensuring it is stable and properly aligned.'}, {'step': 2, 'operation': 'Facing', 'tool_description': '50mm Face Mill', 'tool_diameter_mm': 50, 'spindle_speed_rpm': 1200, 'feed_rate_mm_min': 200, 'notes': 'Face the top surface to achieve a clean, flat finish. Ensure even feed across the surface.'}, {'step': 3, 'operation': 'Drilling', 'tool_description': '6mm Twist Drill Bit', 'tool_diameter_mm': 6, 'spindle_speed_rpm': 1000, 'feed_rate_mm_min': 150, 'notes': 'Drill four M6 through-holes at each corner, located 10mm in from each edge. Use peck drilling if needed for better chip r

In [6]:
# Define our machine and material constraints
MACHINE_CONSTRAINTS = {
    "max_spindle_speed_rpm": 8000,
    "max_feed_rate_mm_min": 5000,
    "valid_operations": {
        "Facing": ["Face Mill"],
        "Drilling": ["Drill Bit", "Center Drill"],
        "Center Drilling": ["Center Drill"],
        "Deburring": ["Deburring Tool", "End Mill"],
    }
}

def validate_plan(plan: list, constraints: dict) -> list:
    """
    Validates each step of the process plan against predefined constraints.

    Args:
        plan: The parsed list of process plan steps.
        constraints: A dictionary containing machine and process limitations.

    Returns:
        The updated plan list with validation results and warning messages.
    """
    validated_plan = []
    for step in plan:
        warnings = []
        
        # 1. Validate spindle speed
        speed = step.get("spindle_speed_rpm")
        if speed and speed > constraints["max_spindle_speed_rpm"]:
            warnings.append(f"Spindle speed of {speed} RPM exceeds machine maximum ({constraints['max_spindle_speed_rpm']} RPM).")
            
        # 2. Validate tool selection
        op = step.get("operation")
        tool = step.get("tool_description")
        if op in constraints["valid_operations"]:
            # Check if the suggested tool contains a valid keyword
            valid_tools = constraints["valid_operations"][op]
            if not any(vt.lower() in tool.lower() for vt in valid_tools):
                warnings.append(f"The suggested tool '{tool}' may be inappropriate for the operation '{op}'. Recommended: {', '.join(valid_tools)}.")

        # 3. We can add more checks here, e.g., for feed rate, material, etc.
        # In our mock data, step 4 has a clear error, which we will highlight manually here.
        if step.get("step") == 4 and speed and speed > 8000:
            warnings.append(f"The speed generated by the LLM ({speed} RPM) for an HSS drill bit is too high. For aluminum with a 5mm diameter, it should typically be in the 2000-4000 RPM range. This is a classic LLM 'hallucination'.")

        # Add warning messages to the step for review
        step["validation_warnings"] = "; ".join(warnings) if warnings else "OK"
        validated_plan.append(step)
        
    return validated_plan

print("\n3. Applying validation logic...")
validated_plan = validate_plan(parsed_plan, MACHINE_CONSTRAINTS)
print("Validation complete.")



3. Applying validation logic...
Validation complete.


In [7]:
print("\n4. Generating final output...")

# Use pandas to create a DataFrame for a formatted table output
df = pd.DataFrame(validated_plan)

# Adjust column order for better readability
desired_order = [
    "step", "operation", "tool_description", "spindle_speed_rpm", 
    "feed_rate_mm_min", "notes", "validation_warnings"
]
# Filter out columns that don't exist in the DataFrame to avoid errors
existing_columns = [col for col in desired_order if col in df.columns]
df = df[existing_columns]

# Print the final process plan table
print("--- LLM-Assisted CNC Process Plan ---")
print(f"Part: {part_description.strip().splitlines()[0]}")
print(f"Material: {part_description.strip().splitlines()[1]}")
print("-" * 30)

# Use to_string() to ensure all content is displayed
print(df.to_string())


4. Generating final output...
--- LLM-Assisted CNC Process Plan ---
Part: Part Name: Simple Mounting Bracket
Material: Material: Aluminum 6061-T6
------------------------------
   step  operation              tool_description  spindle_speed_rpm  feed_rate_mm_min                                                                                                                             notes validation_warnings
0     1      Setup  Vise or fixture for clamping                NaN               NaN                                   Secure the raw stock in a suitable vise or fixture, ensuring it is stable and properly aligned.                  OK
1     2     Facing                50mm Face Mill             1200.0             200.0                                        Face the top surface to achieve a clean, flat finish. Ensure even feed across the surface.                  OK
2     3   Drilling           6mm Twist Drill Bit             1000.0             150.0  Drill four M6 through-holes